In [1]:
import os
import pickle
import numpy as np
import tensorflow as tf
import copy
import time
from matplotlib import pyplot as plt
from joblib import Parallel, delayed
import multiprocessing

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.preprocessing import StandardScaler

from evolution import model

In [2]:
featuremat = np.load('data/training_data/featurematrix.npy')
time_to_failure_vec = np.load('data/training_data/time_to_failure.npy')
testmat = np.load('data/test_processed/featurematrix_test.npy')

In [3]:
trainset = np.random.choice(np.arange(len(time_to_failure_vec)),size=int(len(time_to_failure_vec)*.9),replace=False)
validationset = list(set(np.arange(len(time_to_failure_vec)))-set(trainset))

In [4]:
trainmat = featuremat[trainset,:]
validationmat = featuremat[validationset,:]

time_to_failure_train = time_to_failure_vec[trainset]
time_to_failure_validation = time_to_failure_vec[validationset]

## Normalise data

In [5]:
scaler = StandardScaler()
scaler.fit(trainmat)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [6]:
trainmat_scale = scaler.transform(trainmat)
validationmat_scale = scaler.transform(validationmat)

# Evolution

In [7]:
generations = 100
pop_size = 100
epochs = 100
batch_size = 64
num_all_features = 33

savepath = '/Users/djustus/workspace/earthquake/ckpt/'
model_name = 'E1'

### Create models with one hidden layer (with 1-16 units) each, using 8 random features

In [8]:
model_list = []
for i in range(pop_size):
    model_list.append(model(set(np.random.choice(range(num_all_features),1,replace=False)),
                            [np.random.randint(1,17)],[0],num_all_features))

In [9]:
gen = -1

In [10]:
def train_model(model_list,trainmat,validationmat,model_ind):
    
    trainmat_use = trainmat[:,list(model_list[model_ind].featureset)]
    validationmat_use = validationmat[:,list(model_list[model_ind].featureset)]

    m = model_list[model_ind].build()
    
    opt = tf.keras.optimizers.Adam(lr=0.1,decay=.1)
    m.compile(optimizer=opt,
              loss='mean_squared_error',
              metrics=['mean_absolute_error'])
    hist = m.fit(trainmat_use, 
          time_to_failure_train, 
          validation_data=(validationmat_use,time_to_failure_validation), 
          epochs=epochs, 
          batch_size=batch_size,
          verbose=0
         )
        
    loss = hist.history['val_mean_absolute_error'][-1]
    return loss

In [ ]:
num_cores = multiprocessing.cpu_count()

## Training
Train generation 0 completely.  
Following generation 1: Train only mutations

In [ ]:
loss = np.zeros([generations,pop_size])
models_employed = []
while gen<generations:
    gen+=1
    num_features_gen = []
    num_layers_gen = []
    t = time.time()
    
    models_employed.append(model_list) 
    
    for model_ind in range(pop_size):
        num_features_gen.append(model_list[model_ind].num_features)
        num_layers_gen.append(model_list[model_ind].num_layers)
        
    loss_generation = Parallel(n_jobs=num_cores)(
            delayed(train_model)(model_list,trainmat_scale,validationmat_scale,model_ind) 
            for model_ind in range(int(pop_size/2),pop_size)
    )
#     loss_generation = [train_model(model_list,trainmat_scale,validationmat_scale,model_ind) 
#                        for model_ind in range(pop_size)]
    loss_generation = [1e9 if np.isnan(val) else val for val in loss_generation]
    
    if gen==0:
        loss_survivor = Parallel(n_jobs=num_cores)(
            delayed(train_model)(model_list,trainmat_scale,validationmat_scale,model_ind) 
            for model_ind in range(0,int(pop_size/2))
        )
        loss_survivor = [1e9 if np.isnan(val) else val for val in loss_survivor]
            
    loss[gen,0:int(pop_size/2)] = loss_survivor
    loss[gen,int(pop_size/2):pop_size] = loss_generation
    
    tf.keras.backend.clear_session()    
        
    live = np.argsort(loss[gen,:])[0:int(pop_size/2)]
    best = live[0]
    
    print('Generation %d finished, %.3f sec,\n'
          'average number of features %.1f, average number of layers %.1f, average loss %.3f, \n'          
          'best number of features %.1f, best number of layers %.1f, best loss %.3f'
          %(gen, time.time()-t, 
            np.mean(num_features_gen), np.mean(num_layers_gen), np.mean(loss[gen,:]), 
            model_list[best].num_features, model_list[best].num_layers, loss[gen,best]))
    
    model_list2 = []
    loss_survivor = []
    for model_ind in live:
        model_list2.append(model_list[model_ind])
        loss_survivor.append(loss[gen,model_ind])
    for model_ind in live:
        mutation = copy.deepcopy(model_list[model_ind])
        mutation.mutate()
        model_list2.append(mutation)

    model_list = model_list2
    with open(os.path.join(savepath,'model_list_%s_gen%d.pkl' %(model_name,gen)),'wb') as fn:
        pickle.dump(model_list,fn)

Generation 0 finished, 145.808 sec,
average number of features 1.0, average number of layers 1.0, average loss 3.768, 
best number of features 1.0, best number of layers 1.0, best loss 2.384


/Users/djustus/anaconda3/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Generation 1 finished, 84.714 sec,
average number of features 1.1, average number of layers 1.1, average loss 3.252, 
best number of features 1.0, best number of layers 2.0, best loss 2.230
Generation 2 finished, 76.996 sec,
average number of features 1.1, average number of layers 1.2, average loss 3.172, 
best number of features 1.0, best number of layers 2.0, best loss 2.230
Generation 3 finished, 84.625 sec,
average number of features 1.2, average number of layers 1.2, average loss 3.090, 
best number of features 1.0, best number of layers 2.0, best loss 2.230
Generation 4 finished, 79.382 sec,
average number of features 1.4, average number of layers 1.3, average loss 2.739, 
best number of features 2.0, best number of layers 2.0, best loss 2.216
Generation 5 finished, 89.956 sec,
average number of features 1.4, average number of layers 1.4, average loss 2.661, 
best number of features 2.0, best number of layers 2.0, best loss 2.216
Generation 6 finished, 86.110 sec,
average number 